# Análise de Tópicos

#### Importação de Bibliotecas

In [1]:
import pandas as pd
import spacy
from bertopic import BERTopic

from collections import Counter

import re
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
from gensim.utils import simple_preprocess

import nltk
nltk.download(["stopwords", "rslp"])
stopwords = nltk.corpus.stopwords.words("portuguese")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [3]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForSequenceClassification  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModelForTokenClassification
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline
import torch

#### Leitura do arquivo

O arquivo com os dados da avaliação do restaurante Guacamole, retirados da plataforma TripAdvisor em https://www.tripadvisor.com.br/Restaurant_Review-g303506-d3399400-Reviews-or2000-Guacamole_Cocina_Mexicana_Barra_da_Tijuca-Rio_de_Janeiro_State_of_Rio_de_J.html possui os comentários dos usuários em relação às suas respectivas avaliações do restaurante e a data de tal avaliação.

In [4]:
df_reviews = pd.read_csv("guacamole_reviews.csv")
df_reviews.head()

,Date,Review
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já..."
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest..."
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó..."


In [6]:
df_comments = df_reviews["Review"]
df_dates = df_reviews["Date"]

#### Limpeza dos dados

As funções em seguir estão relacionadas ao pré-processamento de texto em linguagem natural utilizando o português. 
- A função ```remove_stopwords``` realiza a remoção de palavras de parada (stopwords) de textos em português. 
- A função ```lemmatizer``` realiza a lematização de um texto em português, preservando apenas certas classes gramaticais.
- A função ```parse_date``` realiza a conversão de uma data no formato "(dia) de (mês) de (ano)" para "YYYY-MM-DD".

In [7]:
def remove_stopwords(text):
    words = simple_preprocess(text)
    phrase_adjusted = " ".join([word for word in words if word not in stopwords])
    return phrase_adjusted.lower()

spacy_lemma = spacy.load("pt_core_news_sm")

def lemmatizer(text, postags_permit=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    doc = spacy_lemma(text.lower())
    doc_lemma = " ".join([token.lemma_ for token in doc if token.pos_ in postags_permit])
    return doc_lemma

In [8]:
def parse_date(date_str):
    months = {
        'janeiro': 'January', 'fevereiro': 'February', 'março': 'March', 'abril': 'April',
        'maio': 'May', 'junho': 'June', 'julho': 'July', 'agosto': 'August',
        'setembro': 'September', 'outubro': 'October', 'novembro': 'November', 'dezembro': 'December'
    }
    match = re.match(r'(\d+) de (\w+) de (\d+)', date_str)
    if match:
        day, month, year = match.groups()
        month = months[month.lower()]
        return datetime(int(year), list(months.values()).index(month) + 1, int(day)).strftime("%Y-%m-%d")
    else:
        return None

Aplicamos as mudanças nos comentários das avaliações e adicionamos uma nova coluna no DataFrame com os resultados obtidos.

In [9]:
%%time

df_reviews["Review Lemma"] = df_reviews["Review"].map(remove_stopwords)
df_reviews["Review Lemma"] = df_reviews["Review"].map(lemmatizer)

CPU times: total: 15 s
Wall time: 40.9 s


In [9]:
# df_reviews["Date Formatted"] = df_reviews["Date"].apply(parse_date)

In [10]:
df_reviews.head()

,Date,Review,Review Lemma
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...


In [11]:
df_comments = df_reviews["Review Lemma"]
doc_comments = df_comments.to_list()
doc_comments[:5]

['atendimento maravilhoso matheu muito atencioso colocar playlist atendente super simpatico',
 'gostar muito prato atendimento solicito comida saboroso certeza voltar mais vez',
 'ameir bom cozinha mexicano já comi espaço muito animar imersivo decoração casa linda atendimento excelente sempre',
 'bom ambiente decoração funcionário prestativo comida boa música agradável divertido mariachi',
 'amoma atendimento matheu bom amar atendimento sensacional comida muito saboroso']

#### BERTopic

Utilizando os parâmetros padronizados do BERTopic, obteremos os tópicos, do mais frequente ao menos frequente. 
- Se desejamos visualizar a frequência de cada tópico, podemos utilizar ```get_topic_freq()``` para obter os valores.
- Se desejamos visualizar as informações de todos os tópicos obtidos, podemos utilizar ```get_topic_info()``` para obter o número de documentos que estão relacionados a cada tópico e o nome dele, além de suas representações textuais.
- Se desejamos visualizar as informações de um tópico em específico, podemos utilizar o ```get_topic()``` para obter as palavras que pertencem ao tópico e os seus _scores_ de c-TF-IDF, ou seja, uma representação numérica do quão frequente e única aquela palavra é para o documento.

Vale lembrar que o tópico de número **-1** refere-se a todos os valores discrepantes e normalmente deve ser ignorado.

In [15]:
%%time

topic_model = BERTopic(language="portuguese", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(doc_comments)

2024-01-27 15:06:23,002 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/90 [00:00<?, ?it/s]

2024-01-27 15:07:36,304 - BERTopic - Embedding - Completed ✓
2024-01-27 15:07:36,308 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-27 15:07:56,558 - BERTopic - Dimensionality - Completed ✓
2024-01-27 15:07:56,562 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-27 15:07:57,015 - BERTopic - Cluster - Completed ✓
2024-01-27 15:07:57,019 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-27 15:07:57,192 - BERTopic - Representation - Completed ✓


CPU times: total: 5min 45s
Wall time: 1min 34s


In [16]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,330,-1_muito_lugar_bom_atendimento,"[muito, lugar, bom, atendimento, bem, mais, an...",[muito bom divertir amigo família comer comida...
1,0,530,0_mexicano_comida_restaurante_bom,"[mexicano, comida, restaurante, bom, mais, mui...",[atendimento excelente comida muito bom ambien...
2,1,380,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...
3,2,347,2_música_bom_ambiente_muito,"[música, bom, ambiente, muito, musica, atendim...",[atendimento excelente bom comida ambiente div...
4,3,240,3_aniversário_comemorar_amigo_muito,"[aniversário, comemorar, amigo, muito, bom, lu...",[bom atendimento garçom simpático bom lugar ir...


In [17]:
topic_model.get_topic_freq()

,Topic,Count
2,0,530
1,1,380
3,2,347
0,-1,330
10,3,240
4,4,222
9,5,147
13,6,140
15,7,101
5,8,92


In [18]:
topic_model.get_topic(5)

[('restaurante', 0.11182763704088258),
 ('bom', 0.04058356487770332),
 ('muito', 0.03808096625632009),
 ('atendimento', 0.03398111542301553),
 ('comida', 0.029382285371751166),
 ('recomendar', 0.027966993377790735),
 ('bem', 0.02614262085822117),
 ('excelente', 0.024702864872459443),
 ('qualidade', 0.021903869867435956),
 ('super', 0.020857502759721085)]

#### Visualização dos tópicos

In [19]:
topic_model.visualize_barchart(top_n_topics=10)

Com o auxílio do ChatGPT para uma definição em uma única frase sobre o que se trata os primeiros 5 tópicos, temos que:
- **Tópico 0**: Este tópico analisado aborda aspectos relacionados a comida mexicana, com ênfase em guacamole, destacando a qualidade de restaurantes e a atmosfera agradável.
- **Tópico 1**: Este tópico analisado destaca a qualidade do atendimento e do ambiente, ressaltando a excelência do lugar, com ênfase em uma experiência gastronômica muito boa e maravilhosa.
- **Tópico 2**: Esse tópico analisado aborda a influência positiva da música no ambiente, ressaltando a qualidade do atendimento e a excelência, criando uma atmosfera agradável para comer e animar.
- **Tópico 3**: Esse tópico analisado trata da celebração de aniversários e comemorações em um ambiente amigável, destacando a importância do lugar, a qualidade do ambiente e do atendimento, tornando a experiência muito boa para amigos e amigas.
- **Tópico 4**: Este tópico analisado destaca a apreciação de bebidas, especialmente drinks e cervejas, ressaltando a importância do atendimento e a qualidade do ambiente, proporcionando uma experiência muito boa para comer e bebidas geladas.

In [20]:
topic_model.visualize_hierarchy()

In [18]:
# dates_reviews = df_reviews["Date Formatted"].to_list()

In [19]:
# topics_over_time = topic_model.topics_over_time(doc_comments, dates_reviews, datetime_format="%Y-%m-%d", nr_bins=20)

In [20]:
# topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

#### DataFrame com informações completas

In [21]:
df_topics = df_reviews.copy()

df_topics["Topics"] = topics

topic_name = freq.drop(columns=["Count"]).rename(columns={"Topic": "Topics", "Name": "Names"})
df_topics = df_topics.merge(topic_name, how="left")

df_topics.head()

,Date,Review,Review Lemma,Topics,Names,Representation,Representative_Docs
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,-1,-1_muito_lugar_bom_atendimento,"[muito, lugar, bom, atendimento, bem, mais, an...",[muito bom divertir amigo família comer comida...
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,1,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,0,0_mexicano_comida_restaurante_bom,"[mexicano, comida, restaurante, bom, mais, mui...",[atendimento excelente comida muito bom ambien...
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2,2_música_bom_ambiente_muito,"[música, bom, ambiente, muito, musica, atendim...",[atendimento excelente bom comida ambiente div...
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,1,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...


# Análise de Sentimentos

Utilizando um modelo pré-treinado público baseado no BERT, faremos a extração das informações dos sentimentos de cada uma das avaliações.

In [22]:
%%time

model = AutoModelForSequenceClassification.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student')
tokenizer = AutoTokenizer.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student', do_lower_case=False)
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

CPU times: total: 4.02 s
Wall time: 6.1 s


In [23]:
sentiment_task("Eu sou feliz")

[[{'label': 'positive', 'score': 0.9528123140335083},
  {'label': 'neutral', 'score': 0.034104324877262115},
  {'label': 'negative', 'score': 0.013083375990390778}]]

#### Classificação das avaliações e salvamento dos resultados no DataFrame

Para a extração dos sentimentos e gravação das informações dos valores no DataFrame, utilizaremos duas funções:
- Para captura dos sentimentos e gravação no DataFrame na forma padrão de entrega dos resultados, chamaremos primeiro a pipeline do processo, chamada ```sentiment_task``` e armazenaremos os valores em uma coluna chamada "Sentiments".
- Feito isso, iremos separar os valores de acordo com o sentimento e armazenaremos cada um deles no DataFrame final utilizando a função ```extract_sentiment_scores()```.

In [24]:
def extract_sentiment_scores(predictions, sentiment_label):
    if isinstance(predictions, list) and predictions:
        for item in predictions[0]:
            if item['label'] == sentiment_label:
                return item['score']
    return 0

In [25]:
%%time

df_topics["Sentiments"] = df_topics["Review"].apply(lambda x: sentiment_task(x))

CPU times: total: 19min 5s
Wall time: 4min 12s


In [26]:
df_topics["Positive"] = df_topics["Sentiments"].apply(lambda x: extract_sentiment_scores(x, 'positive'))
df_topics["Neutral"] = df_topics["Sentiments"].apply(lambda x: extract_sentiment_scores(x, 'neutral'))
df_topics["Negative"] = df_topics["Sentiments"].apply(lambda x: extract_sentiment_scores(x, 'negative'))

df_topics = df_topics.drop(columns=["Sentiments"])

In [27]:
df_topics.head()

,Date,Review,Review Lemma,Topics,Names,Representation,Representative_Docs,Positive,Neutral,Negative
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,-1,-1_muito_lugar_bom_atendimento,"[muito, lugar, bom, atendimento, bem, mais, an...",[muito bom divertir amigo família comer comida...,0.989543,0.006004,0.004453
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,1,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...,0.626670,0.077070,0.296260
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,0,0_mexicano_comida_restaurante_bom,"[mexicano, comida, restaurante, bom, mais, mui...",[atendimento excelente comida muito bom ambien...,0.967855,0.019304,0.012841
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2,2_música_bom_ambiente_muito,"[música, bom, ambiente, muito, musica, atendim...",[atendimento excelente bom comida ambiente div...,0.940317,0.025686,0.033997
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,1,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...,0.979101,0.012174,0.008725


In [28]:
df_topics.to_csv('guacamole_topics_sentiment.csv', index=False)

Feito isso, iremos calcular a média dos valores de cada um dos três sentimentos para os tópicos em questão e, no final de tudo, veremos qual é o maior valor dentre os três para definirmos o sentimento final para o tópico analisado.

In [29]:
df_topic_sentiment = df_topics.groupby('Topics').agg({'Neutral': 'mean', 'Positive': 'mean', 'Negative': 'mean'})
df_topic_sentiment = df_topic_sentiment.reset_index()
df_topic_sentiment

,Topics,Neutral,Positive,Negative
0,-1,0.051176,0.810901,0.137923
1,0,0.055269,0.820337,0.124395
2,1,0.033215,0.881016,0.085768
3,2,0.037163,0.890303,0.072534
4,3,0.043821,0.840673,0.115506
5,4,0.033489,0.892801,0.073711
6,5,0.040362,0.866447,0.093191
7,6,0.062658,0.789517,0.147825
8,7,0.047895,0.808617,0.143488
9,8,0.032555,0.880048,0.087397


In [30]:
score_cols = ['Neutral', 'Positive', 'Negative']
df_topic_sentiment['Highest Score'] = df_topic_sentiment[score_cols].max(axis=1)

def get_sentiment(row):
    if row['Positive'] == row['Highest Score']:
        return 'Positive'
    elif row['Negative'] == row['Highest Score']:
        return 'Negative'
    else:
        return 'Neutral'

df_topic_sentiment['Topic Sentiment'] = df_topic_sentiment.apply(get_sentiment, axis=1)
df_topic_sentiment

,Topics,Neutral,Positive,Negative,Highest Score,Topic Sentiment
0,-1,0.051176,0.810901,0.137923,0.810901,Positive
1,0,0.055269,0.820337,0.124395,0.820337,Positive
2,1,0.033215,0.881016,0.085768,0.881016,Positive
3,2,0.037163,0.890303,0.072534,0.890303,Positive
4,3,0.043821,0.840673,0.115506,0.840673,Positive
5,4,0.033489,0.892801,0.073711,0.892801,Positive
6,5,0.040362,0.866447,0.093191,0.866447,Positive
7,6,0.062658,0.789517,0.147825,0.789517,Positive
8,7,0.047895,0.808617,0.143488,0.808617,Positive
9,8,0.032555,0.880048,0.087397,0.880048,Positive


In [31]:
df_topic_sentiment.to_csv('topics_sentiment.csv', index=False)

# Extração de Entidade Nomeada

In [32]:
model_ner = AutoModelForTokenClassification.from_pretrained('51la5/roberta-large-NER')
tokenizer_ner = AutoTokenizer.from_pretrained('51la5/roberta-large-NER', do_lower_case=False)
ner_task = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner)

Some weights of the model checkpoint at 51la5/roberta-large-NER were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
ner_task("Julia não gosta de Londres nem Berlim")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'I-PER',
  'score': 0.99990416,
  'index': 1,
  'word': '▁Julia',
  'start': 0,
  'end': 5},
 {'entity': 'I-LOC',
  'score': 0.9999987,
  'index': 5,
  'word': '▁Londres',
  'start': 19,
  'end': 26},
 {'entity': 'I-LOC',
  'score': 0.9999982,
  'index': 7,
  'word': '▁Ber',
  'start': 31,
  'end': 34},
 {'entity': 'I-LOC',
  'score': 0.99999785,
  'index': 8,
  'word': 'lim',
  'start': 34,
  'end': 37}]

In [34]:
def classify_ner(text):
    """
    Token classification function using a pretrained model.

    Parameters:
    - text: Input text to be tokenized and classified.

    Returns:
    List of tuples containing predicted pairs (token, label) for the input text.

    Example:
    Input:  "Julia is tired of living in London."
    Output: [('Julia', 'B-PESSOA'), ('is', 'O'), ('tired', 'O'), ('of', 'O'),
            ('living', 'O'), ('in', 'O'), ('London', 'B-LOCAL'), ('.', 'O')]
    """

    inputs = tokenizer_ner(text, max_length=512, truncation=True, return_tensors="pt")
    tokens = inputs.tokens()

    outputs = model_ner(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)

    results = []

    for token, prediction in zip(tokens, predictions[0].numpy()):
        label = model_ner.config.id2label.get(prediction, 'O')
        if label != 'O':
            results.append((token, label))

    return results

In [35]:
%%time

df_topics["Token Predictions"] = df_topics["Review"].apply(classify_ner)

CPU times: total: 1h 51min 12s
Wall time: 25min 3s


In [36]:
def merge_tokens(token_predictions):
    """
    Function to merge consecutive tokens that start with "_" and have the same label.

    Parameters:
    - token_predictions: List of tuples containing predicted pairs (token, label).

    Returns:
    List of merged tuples where consecutive tokens starting with "_"
    and having the same label are combined.

    Example:
    Input:  [('▁Mathe', 'I-PER'), ('us', 'I-PER')]
    Output: [('▁Matheus', 'I-PER')]
    """

    merged_results = []
    current_token = ""
    current_label = ""

    for token, label in token_predictions:
        if token.startswith("▁"):
            if current_token:
                merged_results.append((current_token, current_label))
            current_token = token[1:]
            current_label = label
        else:
            current_token += token
            current_label = label

    if current_token:
        merged_results.append((current_token, current_label))

    return merged_results

In [37]:
%%time

df_topics["Token Predictions Corrected"] = df_topics["Token Predictions"].apply(merge_tokens)

CPU times: total: 15.6 ms
Wall time: 17.3 ms


In [38]:
token_predictions_corrected = df_topics["Token Predictions Corrected"]

flat_list = [item for sublist in token_predictions_corrected for item in sublist]

counter = Counter(flat_list)

most_common_elements = counter.most_common(20)

most_common_elements

[(('mexicana', 'I-MISC'), 332),
 (('mexicano', 'I-MISC'), 143),
 (('Rio', 'I-LOC'), 86),
 (('México', 'I-LOC'), 65),
 (('Guacamole', 'I-ORG'), 61),
 (('Guacamole', 'I-LOC'), 59),
 (('Barra', 'I-LOC'), 53),
 (('de', 'I-LOC'), 36),
 (('Janeiro', 'I-LOC'), 29),
 (('Jardim', 'I-LOC'), 28),
 (('<s>', 'I-LOC'), 27),
 (('Gabriel', 'I-PER'), 27),
 (('Botânico', 'I-LOC'), 24),
 (('Maria', 'I-MISC'), 23),
 (('Guacamole', 'I-MISC'), 23),
 (('Matheus', 'I-PER'), 21),
 (('Maria', 'I-PER'), 21),
 (('Tijuca', 'I-LOC'), 20),
 (('da', 'I-LOC'), 20),
 (('Mariachis', 'I-ORG'), 17)]

In [39]:
df_topics = df_topics.drop(columns=["Token Predictions"])

In [40]:
df_topics.head()

,Date,Review,Review Lemma,Topics,Names,Representation,Representative_Docs,Positive,Neutral,Negative,Token Predictions Corrected
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,-1,-1_muito_lugar_bom_atendimento,"[muito, lugar, bom, atendimento, bem, mais, an...",[muito bom divertir amigo família comer comida...,0.989543,0.006004,0.004453,"[(Matheus, I-PER)]"
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,1,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...,0.626670,0.077070,0.296260,[]
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,0,0_mexicano_comida_restaurante_bom,"[mexicano, comida, restaurante, bom, mais, mui...",[atendimento excelente comida muito bom ambien...,0.967855,0.019304,0.012841,"[(mexicana, I-MISC)]"
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2,2_música_bom_ambiente_muito,"[música, bom, ambiente, muito, musica, atendim...",[atendimento excelente bom comida ambiente div...,0.940317,0.025686,0.033997,[]
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,1,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...,0.979101,0.012174,0.008725,"[(Matheus, I-PER)]"


In [41]:
import pandas as pd
from collections import Counter

df_filtered = df_topics[df_topics["Token Predictions Corrected"].apply(lambda x: bool(x))]

def most_common_value(lst):
    counter = Counter(lst)
    if counter:
        most_common = counter.most_common(1)[0]
        return most_common[0], most_common[1]
    else:
        return None, 0

result = df_filtered.groupby("Topics")["Token Predictions Corrected"].agg(lambda x: most_common_value([item for sublist in x for item in sublist])).reset_index()
result

,Topics,Token Predictions Corrected
0,-1,"((Rio, I-LOC), 7)"
1,0,"((mexicana, I-MISC), 308)"
2,1,"((Maria, I-PER), 5)"
3,2,"((Rio, I-LOC), 4)"
4,3,"((Barra, I-LOC), 4)"
5,4,"((Eveline, I-PER), 4)"
6,5,"((Maria, I-PER), 4)"
7,6,"((Guacamole, I-ORG), 38)"
8,7,"((<s>, I-LOC), 2)"
9,8,"((Mateus, I-PER), 2)"


In [42]:
result.to_csv('topics_ner.csv', index=False)

### Montagem da planilha final

In [43]:
df_topics.head()

,Date,Review,Review Lemma,Topics,Names,Representation,Representative_Docs,Positive,Neutral,Negative,Token Predictions Corrected
0,4 de dezembro de 2023,Atendimento maravilhoso! Matheus foi muito ate...,atendimento maravilhoso matheu muito atencioso...,-1,-1_muito_lugar_bom_atendimento,"[muito, lugar, bom, atendimento, bem, mais, an...",[muito bom divertir amigo família comer comida...,0.989543,0.006004,0.004453,"[(Matheus, I-PER)]"
1,2 de dezembro de 2023,Gostei muito dos pratos e do atendimento. Muit...,gostar muito prato atendimento solicito comida...,1,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...,0.626670,0.077070,0.296260,[]
2,1 de dezembro de 2023,"Eu amei, é a melhor cozinha mexicana que eu já...",ameir bom cozinha mexicano já comi espaço muit...,0,0_mexicano_comida_restaurante_bom,"[mexicano, comida, restaurante, bom, mais, mui...",[atendimento excelente comida muito bom ambien...,0.967855,0.019304,0.012841,"[(mexicana, I-MISC)]"
3,29 de novembro de 2023,"Ótimo ambiente e decoração, funcionários prest...",bom ambiente decoração funcionário prestativo ...,2,2_música_bom_ambiente_muito,"[música, bom, ambiente, muito, musica, atendim...",[atendimento excelente bom comida ambiente div...,0.940317,0.025686,0.033997,[]
4,27 de novembro de 2023,"Amomamos o atendimento do Matheus e do, tudo ó...",amoma atendimento matheu bom amar atendimento ...,1,1_bom_comida_atendimento_muito,"[bom, comida, atendimento, muito, comer, ambie...",[lugar muito agradável curtir amigo bom atendi...,0.979101,0.012174,0.008725,"[(Matheus, I-PER)]"


In [44]:
df_topic_sentiment.head()

,Topics,Neutral,Positive,Negative,Highest Score,Topic Sentiment
0,-1,0.051176,0.810901,0.137923,0.810901,Positive
1,0,0.055269,0.820337,0.124395,0.820337,Positive
2,1,0.033215,0.881016,0.085768,0.881016,Positive
3,2,0.037163,0.890303,0.072534,0.890303,Positive
4,3,0.043821,0.840673,0.115506,0.840673,Positive


In [45]:
result.head()

,Topics,Token Predictions Corrected
0,-1,"((Rio, I-LOC), 7)"
1,0,"((mexicana, I-MISC), 308)"
2,1,"((Maria, I-PER), 5)"
3,2,"((Rio, I-LOC), 4)"
4,3,"((Barra, I-LOC), 4)"


In [63]:
df_info = df_topics[['Topics', 'Representation']].copy()
df_info = pd.merge(df_info, df_topic_sentiment, on='Topics', how='left')
df_info = pd.merge(df_info, result, on='Topics', how='left')
df_info = df_info.drop_duplicates(subset='Topics')

In [64]:
topic_freq = topic_model.get_topic_freq()
df_info = pd.merge(df_info, topic_freq, left_on='Topics', right_on='Topic', how='left')

In [65]:
df_info = df_info.sort_values(by='Topics')
df_info = df_info.drop(columns='Topic')
df_info.head()

,Topics,Representation,Neutral,Positive,Negative,Highest Score,Topic Sentiment,Token Predictions Corrected,Count
0,-1,"[muito, lugar, bom, atendimento, bem, mais, an...",0.051176,0.810901,0.137923,0.810901,Positive,"((Rio, I-LOC), 7)",330
2,0,"[mexicano, comida, restaurante, bom, mais, mui...",0.055269,0.820337,0.124395,0.820337,Positive,"((mexicana, I-MISC), 308)",530
1,1,"[bom, comida, atendimento, muito, comer, ambie...",0.033215,0.881016,0.085768,0.881016,Positive,"((Maria, I-PER), 5)",380
3,2,"[música, bom, ambiente, muito, musica, atendim...",0.037163,0.890303,0.072534,0.890303,Positive,"((Rio, I-LOC), 4)",347
10,3,"[aniversário, comemorar, amigo, muito, bom, lu...",0.043821,0.840673,0.115506,0.840673,Positive,"((Barra, I-LOC), 4)",240


In [66]:
df_info.to_csv('complete_info_topics.csv', index=False)

In [ ]:
df_topics = df_topics.sort_values(by='Topics')
df_topics = df_topics.drop(columns='Date')
df_topics = df_topics.drop(columns='Names')
df_topics = df_topics.drop(columns='Review Lemma')
df_topics = df_topics.drop(columns='Representation')
df_topics = df_topics.drop(columns='Representative_Docs')

In [73]:
df_topics.head()

,Review,Topics,Positive,Neutral,Negative,Token Predictions Corrected
0,Atendimento maravilhoso! Matheus foi muito ate...,-1,0.989543,0.006004,0.004453,"[(Matheus, I-PER)]"
2072,Muito bom para grupos e comemorações \nBom ate...,-1,0.854495,0.032007,0.113497,[]
1269,"Extremamente divertido, ambiente descontraido....",-1,0.475886,0.076328,0.447787,[]
2192,Local lindamente decorado. Localização com fac...,-1,0.932158,0.027144,0.040698,[]
1266,"Ambiente muito divertido, comida muito boa, at...",-1,0.986336,0.007687,0.005977,[]


In [74]:
df_topic_0 = df_topics[df_topics['Topics'] == 0]
df_topic_1 = df_topics[df_topics['Topics'] == 1]
df_topic_2 = df_topics[df_topics['Topics'] == 2]
df_topic_3 = df_topics[df_topics['Topics'] == 3]
df_topic_4 = df_topics[df_topics['Topics'] == 4]
df_topic_5 = df_topics[df_topics['Topics'] == 5]
df_topic_9 = df_topics[df_topics['Topics'] == 9]

In [75]:
df_topic_0.to_csv('df_topic_0.csv', index=False)
df_topic_1.to_csv('df_topic_1.csv', index=False)
df_topic_2.to_csv('df_topic_2.csv', index=False)
df_topic_3.to_csv('df_topic_3.csv', index=False)
df_topic_4.to_csv('df_topic_4.csv', index=False)
df_topic_5.to_csv('df_topic_5.csv', index=False)
df_topic_9.to_csv('df_topic_9.csv', index=False)